In [1]:
import numpy as np
import open3d as o3d

In [18]:
class ViewerWithCallback:

    def __init__(self, config, device, align_depth_to_color):
        self.flag_exit = False
        self.align_depth_to_color = align_depth_to_color

        self.sensor = o3d.io.AzureKinectSensor(config)
        if not self.sensor.connect(device):
            raise RuntimeError('Failed to connect to sensor')

        color = o3d.camera.PinholeCameraIntrinsicParameters.Kinect2ColorCameraDefault
        depth = o3d.camera.PinholeCameraIntrinsicParameters.Kinect2DepthCameraDefault
        self.intrinsic = o3d.camera.PinholeCameraIntrinsic(color if align_depth_to_color else depth)
        self.extrinsic = o3d.camera.PinholeCameraParameters().extrinsic
            
    def __del__(self):
        del self.sensor
            
    def escape_callback(self, vis):
        self.flag_exit = True
        return False
    
    def run(self):        
        self.flag_exit = False        

        glfw_key_escape = 256
        vis = o3d.visualization.VisualizerWithKeyCallback()
        vis.register_key_callback(glfw_key_escape, self.escape_callback)
        vis.create_window('viewer', 1920, 540)
        print("Sensor initialized. Press [ESC] to exit.")
        
        
        pcd = o3d.geometry.PointCloud()
        
        d = self.sensor.capture_frame(self.align_depth_to_color).depth
        tmp = o3d.geometry.PointCloud.create_from_depth_image(d, self.intrinsic)
        vis.add_geometry(tmp, True)
        vis.remove_geometry(tmp, False)
        
        vis_geometry_added = False
        while not self.flag_exit:
            rgbd = self.sensor.capture_frame(self.align_depth_to_color)
            if rgbd is None:
                continue
            
            if vis_geometry_added:
                vis.remove_geometry(pcd, False)
                vis_geometry_added = False
                
            if not vis_geometry_added:
                pcd = o3d.geometry.PointCloud.create_from_depth_image(rgbd.depth, self.intrinsic)
                vis.add_geometry(pcd, False)
                vis_geometry_added = True

            vis.update_geometry(pcd)
            vis.poll_events()
            vis.update_renderer()

In [20]:
config = o3d.io.AzureKinectSensorConfig()
v = ViewerWithCallback(config, 0, False)

[Open3D INFO] AzureKinectSensor::Connect
[Open3D INFO] sensor_index 0
[Open3D INFO] Serial number: 000489792412
[Open3D INFO] Firmware build: Rel
[Open3D INFO] > Color: 1.6.108
[Open3D INFO] > Depth: 1.6.79[6109.7]


In [21]:
v.run()

Sensor initialized. Press [ESC] to exit.


In [22]:
# Reset viewer
del v